In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from utils.costumelayers import MyDenseLayer, MyConv2DLayer
from utils.downloadmodel import download
import os
from scipy.misc import imread  
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from PIL import Image

In [2]:
image = Image.open(r"picture\poodle.png")
image1= np.array(image).astype(np.float32)
image2 = image1[:,:,:3]
image3 = image2 - np.mean(image2)
image3 = image3.reshape([-1, 227, 227 ,3])

In [3]:
#im1 = (imread(r"picture\poodle.png")[:,:,:3]).astype(np.float32)
#im1 = im1 - np.mean(im1)
#im1[:, :, 0], im1[:, :, 2] = im1[:, :, 2], im1[:, :, 0]
#im1 = im1.reshape([-1, 227, 227, 3])

In [4]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for i in gpus:
    tf.config.experimental.set_memory_growth(device=i, enable=True)
tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)

VirtualDeviceConfiguration(memory_limit=2048)

**if download is slow, just go to http://www.cs.toronto.edu/~guerzhoy/tf_alexnet, and download the file**

In [5]:
path = r'bvlc_alexnet.npy'
if  not os.path.exists(path):
    downloadmodel.download('http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy', path)

In [6]:
data = np.load(path, allow_pickle=True, fix_imports=True, encoding='latin1')
data_1 = data.item()
keys = sorted(data_1.keys())

In [8]:
alex_input = keras.Input([227, 227, 3], dtype=tf.float32)
#alex_image = tf.constant(image3, dtype=tf.float32)
weights = tf.constant(data_1['conv1'][0], dtype=tf.float32)
biases = tf.constant(data_1['conv1'][1], dtype=tf.float32)
conv1_layer = MyConv2DLayer(kernels=weights,
                      biases=biases,
                      activation='relu', 
                      padding='VALID',
                      strides=[4, 4])
conv1 = conv1_layer(alex_input)
conv1_lrn = tf.nn.local_response_normalization(conv1, depth_radius=2, bias=1, alpha=2e-05, beta=0.75)
conv1_max_pool = keras.layers.MaxPool2D(pool_size=(3, 3),strides=[2, 2],padding='valid')(conv1_lrn)
conv1_max_pool_1,conv1_max_pool_2 = tf.split(conv1_max_pool, 2, axis=-1)

In [9]:
weights2_1,weights2_2 = tf.split(tf.constant(data_1['conv2'][0], dtype=tf.float32), 2, axis=-1)
biases2_1, biases2_2= tf.split(tf.constant(data_1['conv2'][1], dtype=tf.float32), 2, axis=-1)
conv2_layer1 = MyConv2DLayer(kernels=weights2_1,
                        biases=biases2_1,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv2_1 =conv2_layer1(conv1_max_pool_1)
conv2_layer2 = MyConv2DLayer(kernels=weights2_2,
                        biases=biases2_2,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv2_2 = conv2_layer2(conv1_max_pool_2)
conv2 = tf.concat([conv2_1, conv2_2], axis=-1)
conv2_lrn = tf.nn.local_response_normalization(conv2, depth_radius=2, bias=1, alpha=2e-05, beta=0.75)
conv2_maxpool = keras.layers.MaxPool2D(pool_size=(3, 3),strides=[2, 2],padding='valid')(conv2_lrn)

In [10]:
weights3 = tf.constant(data_1['conv3'][0])
bias3 = tf.constant(data_1['conv3'][1])
conv3_layer = MyConv2DLayer(kernels=weights3,
                      biases=bias3,
                      strides=[1, 1],
                      padding='SAME',
                      activation='relu')
conv3 = conv3_layer(conv2_maxpool)

In [11]:
conv3_1,conv3_2 = tf.split(conv3, 2, axis=-1)
weight4_1, weight4_2 = tf.split(tf.constant(data_1['conv4'][0], dtype=tf.float32),2, axis=-1)
bias4_1, bias4_2 = tf.split(tf.constant(data_1['conv4'][1], dtype=tf.float32),2, axis=-1)
conv4_layer1 = MyConv2DLayer(kernels=weight4_1,
                        biases=bias4_1,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv4_1 = conv4_layer1(conv3_1)
conv4_layer2 = MyConv2DLayer(kernels=weight4_2,
                        biases=bias4_2,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv4_2 = conv4_layer2(conv3_2)

In [12]:
weight5_1, weight5_2 = tf.split(tf.constant(data_1['conv5'][0], dtype=tf.float32),2, axis=-1)
bias5_1, bias5_2 = tf.split(tf.constant(data_1['conv5'][1], dtype=tf.float32),2, axis=-1)
conv5_layer1 = MyConv2DLayer(kernels=weight5_1,
                        biases=bias5_1,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv5_1 = conv5_layer1(conv4_1)
conv5_layer2 = MyConv2DLayer(kernels=weight5_2,
                        biases=bias5_2,
                        strides=[1, 1],
                        padding='SAME',
                        activation='relu')
conv5_2 = conv5_layer2(conv4_2)
conv5 = tf.concat([conv5_1, conv5_2], axis=-1)
conv5_maxpool = keras.layers.MaxPool2D(pool_size=(3, 3),strides=[2, 2],padding='valid')(conv5)

In [13]:
flatten = keras.layers.Flatten()(conv5_maxpool)

In [14]:
weight6 = tf.constant(data_1['fc6'][0], dtype=tf.float32)
biases6 = tf.constant(data_1['fc6'][1], dtype=tf.float32)
fc_layer1 = MyDenseLayer(weights=weight6,
                    biases=biases6,
                    activation='relu')
fcn1 = fc_layer1(flatten)

In [15]:
weight7 = tf.constant(data_1['fc7'][0], dtype=tf.float32)
biases7 = tf.constant(data_1['fc7'][1], dtype=tf.float32)
fc_layer2 = MyDenseLayer(weights=weight7,
                    biases=biases7,
                    activation='relu')
fcn2 = fc_layer2(fcn1)

In [16]:
weight8 = tf.constant(data_1['fc8'][0], dtype=tf.float32)
biases8 = tf.constant(data_1['fc8'][1], dtype=tf.float32)
fc_layer3 = MyDenseLayer(weights=weight8,
                    biases=biases8,
                    activation='softmax')
out = fc_layer3(fcn2)

In [19]:
model = keras.Model(inputs=alex_input, outputs=out)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
my_conv2d_layer_1 (MyConv2DLaye (None, 55, 55, 96)   34944       input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_LRN (TensorFlowOpLa [(None, 55, 55, 96)] 0           my_conv2d_layer_1[0][0]          
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 27, 27, 96)   0           tf_op_layer_LRN[0][0]            
______________________________________________________________________________________________

In [23]:
output = model.predict(image3)

In [24]:
output1 = output[0]
output2 = np.argsort(output1)

In [25]:
output2[:5]

array([986, 957, 352, 926, 576], dtype=int64)

In [26]:
output2[-5:]

array([228, 181, 267, 265, 266], dtype=int64)

In [29]:
model.save(r'alexnet\1')